In [2]:
# loading all the dependencies
import pandas as pd
from openpyxl import load_workbook
import numpy as np
import uncertainties as uc

pd.options.display.float_format ='{:,.2f}'.format
import matplotlib.pyplot as plt
%matplotlib inline  

import scipy.stats as stats
from uncertainties import ufloat
from uncertainties import unumpy


# Adipocytes
Adipocytes, also known as lipocytes and fat cells, are the cells that primarily compose adipose tissue, specialized in storing energy as fat.

Our estimate rely on [Spalding et al., 2008](https://www.nature.com/articles/nature06902), who measured the number of adipocytes in adult humans, their mean volume and their turnover, based on the average age of adipocytes measured by C14 carbon. 

## Number and mass of adipocytes
[Spalding et al., 2008](https://www.nature.com/articles/nature06902) estimated the fat contents of the subjects according to their BMI. They measured the mean adipocyte volume for each subject and calculated the total number of adipocytes by dividing the two.
They divided the subject by age group and seperated between lean and obease. 
According to their theory, their is non-linear connection between the number of adipocyes, their average volume and the total fat mass of the subject. They postulate that for lean individual the fat mass increase due to mainly increase in the size of adipocytes, but in obease the increase in mass cause also by increase in the number of adipocytes. As expressed in the equation: 
$$n=n_{0}+k_{f}M$$
Where, $n$ is the number of adipocytes, $n_{0}$ its lower limit and $k_{f}$ the proportial linearly increase adipocytes number with the body .
From this equtaion, follow a saturation function for the mean volume of adipocytes ($V$) in respect to the total fat mass ($M$)  
$$ V= \frac{aM}{1+bM}$$  
Where, $a=(g_{f}n_{0})^{-1}$, and $b=n_{0}^{-1}k_{f}$,$g_{f}$ is the density of body fat, 

We use the parameters they estimated by fitting the model to data of close to 200 men. 

We use the fat content of the ref. 70 kg man, according to [Snyder et al., 1975](http://www.icrp.org/publication.asp?id=ICRP%20Publication%2023) to derive his total fat mas

In [3]:
adipo_data = pd.read_excel('Adipocytes_data.xlsx','adipocyte_volume',index_col=0)
adipo_data.head()

,value,SD,comment,source
parameter,,,,
a,53.00,7.00,(gf n0)^-1,"Spaldin et al., 2008"
b,0.05,0.01,n0-1kf,"Spaldin et al., 2008"
gf,0.90,nan,"in units: g/ml, 0.9*10^12 in kg/pL","Spaldin et al., 2008"
ref. man fat content (%),0.19,0.02,NaN,"Snyder et al., 1975"


In [4]:
ref_man_total_fat = 70*ufloat(adipo_data.loc['ref. man fat content (%)','value'],adipo_data.loc['ref. man fat content (%)','SD'])
#multiplying be the ref. man mass

#using the parmeters and models to estimate the number of adipocytes for the ref man, and the mean volume of an adipocyte
a = ufloat(adipo_data.loc['a','value'],adipo_data.loc['a','SD'])
b = ufloat(adipo_data.loc['b','value'],adipo_data.loc['b','SD'])
gf = adipo_data.loc['gf','value']

#estimating the number of adipocytes using the equation from above
n0= 1/(a*gf)*10**12 #because gf is given in the source in units of kg/pL 
kf = b*n0

adipocyte_num = n0+kf*ref_man_total_fat
print('Mean number of adipocytes in the ref. man {:0.1eP} cells'.format(adipocyte_num))

#using the equation for adipocyte volume:
mean_adi_volume = a*ref_man_total_fat/(1+b*ref_man_total_fat)  #in pico liter. 
print('Mean volume of adipocyte in the ref. man {:0.0fP} pL'.format(mean_adi_volume))

#using the density of fat we can estimate the mass of an adipocyte from its volume 
mean_adi_mass = mean_adi_volume*gf*1000 #converting from pico L to cm^3
print('Mean mass of adipocyte in the ref. man {:0.0eP} pg'.format(mean_adi_mass))


total_adipo_mass = mean_adi_mass*adipocyte_num/10**12
print('Total cellular mass of adipocyte in the ref. man {:0.0fP} g'.format(total_adipo_mass))

Mean number of adipocytes in the ref. man (3.4±0.5)×10¹⁰ cells
Mean volume of adipocyte in the ref. man 437±74 pL
Mean mass of adipocyte in the ref. man (4±1)×10⁵ pg
Total cellular mass of adipocyte in the ref. man 13300±1400 g


## Lifespan and turnover rate of adipocytes
[Spalding et al., 2008](https://www.nature.com/articles/nature06902) measured the c14 levels in the adipocytes and determine their age by comparison to c14 basic levels in the atmosphere.
They fitted the cell age data to three models for the dynamic of the population, assuming that there is a constant death rate (in some of them they assumed that the population is fixed and other the assumed that the production rate could be different). The death rate estimates were very close in the all the different model. We choose to use the simplest one - constant production and constnat death rate. 

We can derive the mean lifespan of an adipocyte by taking 1 over the turnover rate. 


In [5]:
#loading the relevnat parameters according to Spalding et al., 2008
adipo_data = pd.read_excel('Adipocytes_data.xlsx','number_and_turnover',index_col=0)
adipo_data.head()

,value,SD,comment
parameter,,,
turnover rate [% /year],0.08,0.05,"for lean. Took median +- average deviation, as..."
mean age [years],9.90,3.50,lean population


In [6]:
#turnover rate was given as % of cells that are being replace every year, not affected by age
adi_to_rate =ufloat(adipo_data.loc['turnover rate [% /year]','value'],adipo_data.loc['turnover rate [% /year]','SD']) 

#multiplying the two values to get the cellular turonver rate in cells per day
cellular_to_rate = adi_to_rate*adipocyte_num/365

print("Adipocytes turnover is {:0.1fP}% per year".format(100*adi_to_rate))
print("Adipocytes cellular turnover rate is {:0.1eP} cells per year".format(365*cellular_to_rate))
print("Adipocytes cellular turnover rate is {:0.1eP} cells per day".format(cellular_to_rate))
print('_________________')

#lifespan can be derive from turnover rate (in %) by 1/
adi_lifespan = 1/adi_to_rate #in years units
print("Adipocytes mean lifespan is {:0.1fP} years".format(adi_lifespan))


Adipocytes turnover is 8.2±5.3% per year
Adipocytes cellular turnover rate is (2.8±1.8)×10⁹ cells per year
Adipocytes cellular turnover rate is (7.6±5.1)×10⁶ cells per day
_________________
Adipocytes mean lifespan is 12.2±7.9 years


### Cellular mass turnover
We now can integrate the mass and turnover rate data to get a cellular mass turnover rate estimate:  

In [7]:
cellular_mass_to_rate = cellular_to_rate*mean_adi_mass/10**12 #moving from pf to grams

print("Adipocytes cellular mass turnover rate is {:0.1eP} grams per year".format(cellular_mass_to_rate*365))
print("Adipocytes cellular mass turnover rate is {:0.1fP} grams per day".format(cellular_mass_to_rate))


Adipocytes cellular mass turnover rate is (1.1±0.7)×10³ grams per year
Adipocytes cellular mass turnover rate is 3.0±2.0 grams per day


## Saving the results to excell
So it can be used for summary 

In [9]:
results =pd.DataFrame(index = range(4), columns=['Parameter','Value','Units','Uncertainty']) 
results.loc[0] = pd.Series({
                'Parameter': 'number',
                'Value': adipocyte_num.nominal_value,
                'Units': 'Cells',
                'Uncertainty': adipocyte_num.std_dev
                })

results.loc[1] = pd.Series({
                'Parameter': 'lifespan',
                'Value': 365*adi_lifespan.nominal_value, #from years to days
                'Units': 'days',
                'Uncertainty': 365*adi_lifespan.std_dev #from years to days
                })

results.loc[2] = pd.Series({
                'Parameter': 'cellular turnovr rate',
                'Value': cellular_to_rate.nominal_value,
                'Units': 'cells per day',
                'Uncertainty':cellular_to_rate.std_dev
                })

results.loc[3] = pd.Series({
                'Parameter': 'cell mass',
                'Value': mean_adi_mass.nominal_value,
                'Units': 'pg',
                'Uncertainty':mean_adi_mass.std_dev
                })

results.loc[4] = pd.Series({
                'Parameter': 'cellular mass turnovr rate',
                'Value':cellular_mass_to_rate.nominal_value,
                'Units': 'grams per day',
                'Uncertainty': cellular_mass_to_rate.std_dev
                })

results.loc[5] = pd.Series({
                'Parameter': 'total cellular mass',
                'Value':total_adipo_mass.nominal_value,
                'Units': 'grams per day',
                'Uncertainty': total_adipo_mass.std_dev
                })

%run Utility_Functions.ipynb
#results
file = 'Summary.xlsx'
book = load_workbook(file)
writer = pd.ExcelWriter(file, engine = 'openpyxl')
writer.book = book

writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
sheet=writer.book.active

results.to_excel(writer, sheet_name = 'Adipocytes',index=False)
worksheet = writer.sheets['Adipocytes']   
#     using the colors defined in the color sheet to color the tabs. Using the extraction of the data in the Utility_Functions.ipynb notebook
worksheet.sheet_properties.tabColor= colors_data.loc['Adipocytes','hex'] 


writer.save()
